In [258]:
%reset -f

In [259]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('classic')

%matplotlib inline

In [260]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    if "summary" not in str(f):
        ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    
dfAll = pd.concat(ddf)
print dfAll.shape

(60, 27)


In [261]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_rec_mean = pd.to_numeric(dfAll.cv_rec_mean)
dfAll.cv_prec_mean = pd.to_numeric(dfAll.cv_prec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_prec_std = pd.to_numeric(dfAll.cv_prec_std)
dfAll.cv_rec_std = pd.to_numeric(dfAll.cv_rec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(2).astype(str).str.cat(dfAll["cv_f1_std"].round(2).astype(str), sep="+/-")
dfAll["cv_prec"] = dfAll["cv_prec_mean"].round(2).astype(str).str.cat(dfAll["cv_prec_std"].round(2).astype(str), sep="+/-")
dfAll["cv_rec"] = dfAll["cv_rec_mean"].round(2).astype(str).str.cat(dfAll["cv_rec_std"].round(2).astype(str), sep="+/-")
dfAll["num_nones"] = np.sum(np.hstack(((dfAll.sm == "none").reshape(-1,1),(dfAll.fs == "none").reshape(-1,1))), axis=1)

print dfAll.columns.tolist()
print dfAll.shape

['exp', 'typeDisease', 'typeEncounter', 'typeHypothesis', 'typeDataFeatures', 'typeDiagnosis', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_f1', 'tr_prec', 'tr_rec', 'cv_f1_mean', 'cv_f1_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'test_f1', 'test_prec', 'test_rec', 'test_auc', 'time', 'pipeline', 'cv_f1', 'cv_prec', 'cv_rec', 'num_nones']
(60, 31)


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


In [263]:
typeHypothesis = "early_readmission_vs_none"
typeDataFeatures = "extended_extra"
typeDiagnosis = "diag_1"

In [264]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)

filterCond =  np.logical_and(np.logical_and( dfAll['typeDataFeatures'] == typeDataFeatures, 
                                             dfAll['typeHypothesis'] == typeHypothesis),
                                             dfAll['typeDiagnosis'] == typeDiagnosis)
dfAll[filterCond][[
    "typeDataFeatures",'typeHypothesis',"metric","cls","params",'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean','test_f1', 'test_prec', 'test_rec', 'test_auc']]

,typeDataFeatures,typeHypothesis,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,extended_extra,early_readmission_vs_none,f1_weighted,logReg,"[None, 1e-05, l2]",0.8216,0.8135,0.8642,0.8203,0.8080,0.8607,0.8222,0.8640,0.8140,0.5306
0,extended_extra,early_readmission_vs_none,f1_weighted,knn,"[uniform, 5]",0.8418,0.8589,0.8790,0.8130,0.7938,0.8592,0.8129,0.8574,0.7929,0.5146
0,extended_extra,early_readmission_vs_none,f1_weighted,rf,"[entropy, None, 350]",1.0000,1.0000,1.0000,0.8105,0.8173,0.8686,0.8108,0.8690,0.8200,0.5044
0,extended_extra,early_readmission_vs_none,recall,knn,"[uniform, 1]",1.0000,1.0000,1.0000,0.7892,0.7852,0.7934,0.7898,0.7953,0.7846,0.5253
0,extended_extra,early_readmission_vs_none,f1,knn,"[uniform, 1]",1.0000,1.0000,1.0000,0.7892,0.7852,0.7934,0.7898,0.7953,0.7846,0.5253
0,extended_extra,early_readmission_vs_none,f1,rf,"[gini, 6, 400]",0.7186,0.8276,0.6659,0.7138,0.8166,0.6610,0.7128,0.6591,0.8206,0.6229
0,extended_extra,early_readmission_vs_none,recall,rf,"[entropy, 6, 100]",0.7189,0.8270,0.6663,0.7138,0.8166,0.6610,0.7128,0.6592,0.8200,0.6213
0,extended_extra,early_readmission_vs_none,f1,logReg,"[balanced, 0.001, l2]",0.7049,0.8230,0.6488,0.6995,0.8190,0.6423,0.7025,0.6458,0.8233,0.6278
0,extended_extra,early_readmission_vs_none,recall,logReg,"[balanced, 1e-05, l2]",0.6582,0.8211,0.5919,0.6565,0.8183,0.5899,0.6559,0.5893,0.8201,0.6121
0,extended_extra,early_readmission_vs_none,recall,nb,[],0.0322,0.8862,0.1319,0.0326,0.8376,0.1320,0.0309,0.1311,0.7124,0.4999


#### Grouped by Diseases

In [189]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)
dfAll.groupby(["typeDisease"])[["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']].first()

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
typeDisease,,,,,,,,,,,,,,,
subset,1.00,none,f1_weighted,rf,"[gini, 12, 200]",0.70,0.70,0.70,0.62,0.62,0.61,0.61,0.61,0.62,0.61


In [161]:
dfAll[dfAll["typeDisease"] == "Neoplasms_1"][["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.40,none,f1_weighted,rf,"[gini, 6, balanced, 100]",0.83,0.83,0.83,0.64,0.63,0.65,0.62,0.63,0.62,0.54
0,0.20,none,f1_weighted,logReg,"[None, 0.0001, l2]",0.66,0.67,0.71,0.64,0.65,0.69,0.63,0.68,0.63,0.54
0,0.60,none,f1_weighted,rf,"[gini, 6, balanced, 250]",0.82,0.82,0.82,0.64,0.63,0.64,0.62,0.62,0.62,0.55
0,0.40,none,recall,rf,"[entropy, 4, balanced, 250]",0.74,0.75,0.74,0.63,0.64,0.62,0.61,0.61,0.62,0.55
0,0.20,none,roc_auc,logReg,"[None, 0.005, l2]",0.64,0.67,0.71,0.63,0.64,0.69,0.63,0.69,0.63,0.53
0,0.40,none,roc_auc,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,recall,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,f1_weighted,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.20,none,f1_weighted,rf,"[gini, 6, balanced, 200]",0.88,0.88,0.88,0.62,0.61,0.62,0.61,0.61,0.60,0.53
0,0.60,none,f1_weighted,logReg,"[balanced, 0.05, l2]",0.63,0.67,0.61,0.61,0.65,0.59,0.57,0.55,0.61,0.54


#### Save results

In [211]:
import time
dfAll.iloc[:,:-4].to_pickle(os.path.join(pathDir, "summary" + time.strftime("%Y%m%d-%H%M%S") + ".pkl"))

In [213]:
dd = pd.read_pickle(os.path.join(pathDir,"summary20171017-171147.pkl"))
print dd.columns

Index([u'exp', u'typeDisease', u'typeEncounter', u'typeHypothesis',
       u'typeDataFeatures', u'typeDiagnosis', u'size_tr', u'fs', u'sm', u'cls',
       u'metric', u'params', u'tr_f1', u'tr_prec', u'tr_rec', u'cv_f1_mean',
       u'cv_f1_std', u'cv_prec_mean', u'cv_prec_std', u'cv_rec_mean',
       u'cv_rec_std', u'test_f1', u'test_prec', u'test_rec', u'test_auc',
       u'time', u'pipeline'],
      dtype='object')
